# Welcome to the microlensing data challenge tutorial

This notebook is intended to provide the framework for non-microlensers to participate in the microlensing data challenge ( http://microlensing-source.org/data-challenge/ ). It requires installation of the pyLIMA, MulensModel and muLAn modules. 

The data presented in this tutorial is from the Sagan workshop, thus we know the "true" parameter values. For an introduction to microlensing data and analysis, please review the slides and hands on session materials from the 2017 Sagan Workshop ( http://nexsci.caltech.edu/workshop/2017/ ). 

See the data challenge website for the real data, and replace any of the parameter inputs, as necessary for the type of light curve.

This notebook ingests initial guesses for the input WFIRST light curve, creates a model light curve using pyLIMA, MulensModel and muLAn, measures the goodness of fit of the model to the data, and recalculates the model for a few steps within a specified grid range. It calculates the chi-squared at all these positions and determines the minimum. 

To de-bug, uncomment print and plot statements.

Note: The scipy.stats.chisquare() function is intended as a placeholder for the user to replace with a goodness of fit algorithm of their choosing.

If you would like an introduction to python notebooks, please read this tutorial: https://medium.com/codingthesmartway-com-blog/getting-started-with-jupyter-notebook-for-python-4e7082bd5d46

In [ ]:
####If you want interactive plots, uncomment this line and add plt.close() after running each plot
#%matplotlib notebook

###Import required libraries
import numpy as np
import scipy.optimize as op
import matplotlib.pyplot as plt
import astropy.units as u
import os
import sys
import time
from scipy.stats import chisquare
from datetime import datetime
from astropy.coordinates import SkyCoord
import ConfigParser as cp
import matplotlib
import pandas as pd
from scipy import stats

##Change this to the path with your microlensing packages.
local_path='/Users/meshkat/python/'

sys.path.append(local_path+'pylima/pyLIMA-master')
from pyLIMA import event
from pyLIMA import telescopes
from pyLIMA import microlmodels
from pyLIMA import microlcaustics
from pyLIMA import microltoolbox

sys.path.append(local_path+'MulensModel/source')
import MulensModel as mm
from MulensModel.utils import Utils

import muLAn.mulan as mulan
import muLAn.models.ephemeris as ephemeris

In [ ]:
###define path to file and filename
filename = local_path+'data_challenge/WFIRST_1827.dat'

WFIRST_data = np.loadtxt(filename)

figure1 = plt.figure()
plt.errorbar(WFIRST_data[:,0]-2460000,WFIRST_data[:,1],yerr = WFIRST_data[:,2],fmt='.k')
plt.xlabel('HJD-2460000',fontsize=20)
plt.ylabel('MAG',fontsize=20)
plt.gca().invert_yaxis()
figure1.show()


In [ ]:
##Replace any infinite or NAN values with the surrounding data
where_inf=np.where(WFIRST_data==np.inf)
WFIRST_data[where_inf]=np.mean((WFIRST_data[where_inf[0][0]-1,where_inf[1][0]],
                                WFIRST_data[where_inf[0][0]+1,where_inf[1][0]]))

In [ ]:
####CAN CHANGE THESE VALUES:
#For changes, we recommend you copy and comment these original values, to prevent losing the correct solutions.
#Some of these parameters can be changed by looking at the plot above
#For definitions of the parameters, see http://nexsci.caltech.edu/workshop/2017/Tuesday_HandsOn_Yee.pptx slide 3.
to = 2460963.9  
uo = 0.45 
tE = 24.3
rho = 0.007
s = 10**0.198
q = 10**-3.23
alpha = -0.5 #######in radians

log_s = np.log10(s)
log_q = np.log10(q)

# Time range of planetary perturbation (including 2460000).
(t_anomaly_start, t_anomaly_stop) = (2460982., 2460985.)

### time when the peak of the planet anomaly occurs (HJD)
t_anomaly = 2460983.0

### duration of the anomaly in days
delta_t = 1



In [ ]:
####CAN CHANGE THESE VALUES:
####How much you want to vary each parameter by, in the grid search:
####The number of grid steps can also be varied per parameter below
grid_steps=5
to_grid=np.linspace(-0.1,0.1,grid_steps)+to
uo_grid=np.linspace(0.95,1.05,grid_steps)*uo
tE_grid=np.linspace(-1,1,grid_steps)+tE
rho_grid=np.linspace(0.95,1.05,grid_steps)*rho
log_s_grid=np.linspace(0.95,1.05,grid_steps)*log_s
log_q_grid=np.linspace(0.95,1.05,grid_steps)*log_q
alpha_grid=np.linspace(-0.1,0.1,grid_steps)+alpha


###For the sake of this example, we will only perform a chi-squared minimization on the tE and uo parameters.

# Pylima

In [ ]:
start_time_pl = datetime.now()

In [ ]:

pl_event = event.Event()


pl_event.name = 'WFIRST_1827'

### We define a telescope object.
wfirst = telescopes.Telescope(name='WFIRST', camera_filter='W149', light_curve_magnitude=WFIRST_data)



pl_event.telescopes.append(wfirst)

### Let's define a binary model, name in pyLIMA is Uniform Source Binary Lens.

binary_model = microlmodels.create_model('USBL', pl_event)
binary_model.define_model_parameters()

full_chi_sq_pylima=np.zeros((grid_steps,grid_steps))
full_model_in_magnitude=np.zeros((grid_steps,grid_steps,WFIRST_data[:-1,0].shape[0]))

##Run loop a small grid on different values of your choice (in this case, uo and tE):
for j,uo_i in enumerate(uo_grid):
    uo0=uo_i
    for k,tE_i in enumerate(tE_grid):
        tE0=tE_i
        
        ### Here are your input parameter guesses:
        estimate_parameters = [to, uo0, tE0, rho, np.log10(s), np.log10(q), alpha]

        pyLIMA_parameters = binary_model.compute_pyLIMA_parameters(estimate_parameters)


        pl_model, wfirst_source_flux, wfirst_blend_flux = binary_model.compute_the_microlensing_model(wfirst, pyLIMA_parameters)
        model_in_magnitude = microltoolbox.flux_to_magnitude(pl_model)  

#         plt.figure()
#         plt.subplot(111)
#         plt.errorbar(WFIRST_data[:,0]-2460000,WFIRST_data[:,1],yerr = WFIRST_data[:,2],fmt='.k')
#         plt.gca().invert_yaxis()
#         plt.plot(wfirst.lightcurve_flux[:,0]-2460000, model_in_magnitude,zorder=10)
#         plt.title('Plot with parameter 1: '+str(uo0)+' and parameter 2: '+str(tE0))
#         plt.show()

        full_chi_sq_pylima[j,k],_=chisquare(WFIRST_data[:-1,1], f_exp=model_in_magnitude)
        full_model_in_magnitude[j,k,:]=model_in_magnitude
    
plt.imshow(full_chi_sq_pylima,origin='bottom',extent=[tE_grid[0],tE_grid[-1],uo_grid[0],uo_grid[-1]],aspect="auto")
plt.xlabel('tE',fontsize=20)
plt.ylabel('uo',fontsize=20)
plt.colorbar(label='Chi-squared')
plt.show()



In [ ]:
min_index=np.unravel_index(full_chi_sq_pylima.argmin(),full_chi_sq_pylima.shape)
uo0 = uo_grid[min_index[0]]
tE0 = tE_grid[min_index[1]]
chi_sq_pylima=np.min(full_chi_sq_pylima)
model_in_magnitude_pylima=full_model_in_magnitude[min_index[0],min_index[1]]
print "\n Grid search space is minimized at uo=",uo_grid[min_index[0]]," and tE=",tE_grid[min_index[1]],'\n'

In [ ]:
end_time_pl=datetime.now()
print('Total Runtime: {0}'.format(end_time_pl - start_time_pl))
t= format(end_time_pl - start_time_pl)
(hours, minutes, seconds) = t.split(':')
hours_to_run_pl = float(hours) + float(minutes) / 60 + float(seconds) / 3600

In [ ]:
ModelID_pl='WFIRST_1827.dat_pylima' #Model ID containing both target ID and solution number e.g. [target]_[solution]
t0_pl = to #Time of peak, t0 [days] - priority
sig_t0_pl = to_grid[1]-to_grid[0] #Uncertainty in t0 [days] - priority
tE_pl = tE0 #Einstein crossing time, tE [days] - priority
sig_tE_pl = tE_grid[1]-tE_grid[0] #Uncertainty in tE [days] - priority
u0_pl = uo0 #Minimum impact parameter, u0 [normalised by θE] - priority
sig_u0_pl =  uo_grid[1]-uo_grid[0] #Uncertainty in u0 - priority
rho_pl = rho #Angular source size parameter, rho
sig_rho_pl = rho_grid[1]-rho_grid[0] #Uncertainty on rho
piEE_pl = 'None'  #Parallax parameter πE,E
sig_piEE_pl = 'None'  #Uncertainty on πE,E
piEN_pl = 'None'  #Parallax parameter πE,N
sig_piEN_pl = 'None'  #Uncertainty on πE,N
fs_pl = 0  #Source flux, fs [counts/s] - priority
sig_fs_pl = 0  #Uncertainty in fs [counts/s] - priority
fb_pl = 0 #Blend flux, fb [counts/s] - priority
sig_fb_pl = 0 #Uncertainty in fb [counts/s] - priority
s_pl = 10**(log_s) #Binary separation, s, [normalised by θE] - priority
sig_s_pl = 10**(log_s_grid[1]-log_s_grid[0]) #Uncertainty on s - priority
q_pl = 10**(log_q) #Mass ratio, q = M2/M1 - priority
sig_q_pl = 10**(log_q_grid[1]-log_q_grid[0]) #Uncertainty on q - priority
alpha_pl = alpha #Angle of lens motion, alpha - priority
sig_alpha_pl = alpha_grid[1]-alpha_grid[0] #Uncertainty on alpha - priority
dsdt_pl = 'None'  #Rate of change of s, ds/dt
sig_dsdt_pl = 'None'  #Uncertainty on ds/dt
dadt_pl =  'None' #Rate of change of alpha, dalpha/dt
sig_dadt_pl = 'None'  #Uncertainty on dalpha/dt
t0_par_pl = 'None'  #t0_par [days]
chisq_pl = chi_sq_pylima #Chi squared of the fitted model
M1_pl = 'None'  #Primary lens mass, M1 [Msolar]
sig_M1_pl = 'None'  #Uncertainty on M1 [Msolar]
M2_pl = 'None'  #Secondary lens mass, M2 [MJupiter] 
sig_M2_pl = 'None'  #Uncertainty on M2 [MJupiter] 
DL_pl = 'None'  #Distance to the lens, DL [kpc]
sig_DL_pl = 'None'  #Uncertainty on DL [kpc]
DS_pl = 'None'  #Distance to the source, DS [kpc]
sig_DS_pl = 'None'  #Uncertainty on DS [kpc]
aperp_pl = 'None'  #Projected separation of lens masses, aperp [AU]
sig_aperp_pl = 'None'  #Uncertainty on aperp [AU]
t_fit_pl = hours_to_run_pl #Time taken to fit the lightcurve from data ingest to final output [hrs]

# MulensModel

In [ ]:
start_time_mlm = datetime.now()

data = mm.MulensData(file_name=filename)

### Uncomment this to view the lightcurve: 
# plt.errorbar(data.time, data.mag, yerr=data.err_mag, fmt='o')
# plt.gca().invert_yaxis()
# plt.show()

In [ ]:
# *Set the magnification methods for the planet model*
# VBBL method will be used between t_planet_start and t_planet_stop, 
# and point_source_point_lens will be used everywhere else.
magnification_methods = [
    0., 'point_source_point_lens', 
    t_anomaly_start, 'VBBL', t_anomaly_stop, 
    'point_source_point_lens', 2470000.]

In [ ]:
data.bad = np.isnan(data.err_mag)

In [ ]:
# Check the estimated model
# Note that there are two possibilities for s: s_plus and s_minus. 

full_chi_sq_mulens=np.zeros((grid_steps,grid_steps))
full_model_in_magnitude=np.zeros((grid_steps,grid_steps,WFIRST_data[:,0].shape[0]))

#Run loop a small grid on different values of your choice (in this case, uo and tE):
for j,uo_i in enumerate(uo_grid):
    uo0=uo_i
    for k,tE_i in enumerate(tE_grid):
        tE0=tE_i

        # Define the model
        mlm_model = mm.Model({
            't_0': to, 
            'u_0': uo0,
            't_E': tE0,
            'rho': rho,
            's': s,
            'q': q,
            'alpha': np.rad2deg(alpha)})

        mlm_model.set_magnification_methods(magnification_methods)
        mlm_event = mm.Event(datasets=data, model=mlm_model)

        times=WFIRST_data[:,0]

        mlm_event.get_chi2()
        flux = mlm_event.fit.get_flux(data)

        model_in_magnitude0=Utils.get_mag_from_flux(flux)
        
#         plt.figure()
#         plt.subplot(111)
#         plt.errorbar(WFIRST_data[:,0]-2460000,WFIRST_data[:,1],yerr = WFIRST_data[:,2],fmt='.k')
#         plt.gca().invert_yaxis()
#         plt.plot(times-2460000, model_in_magnitude0,zorder=10)
#         plt.title('Plot with parameter 1: '+str(uo0)+' and parameter 2: '+str(tE0))
#         plt.show()

        full_chi_sq_mulens[j,k],_=chisquare(WFIRST_data[:,1],f_exp=model_in_magnitude0)
        full_model_in_magnitude[j,k,:]=model_in_magnitude0
        
plt.imshow(full_chi_sq_mulens,origin='bottom',extent=[tE_grid[0],tE_grid[-1],uo_grid[0],uo_grid[-1]],aspect="auto")
plt.xlabel('tE',fontsize=20)
plt.ylabel('uo',fontsize=20)
plt.colorbar(label='Chi-squared')
plt.show()



In [ ]:
min_index=np.unravel_index(full_chi_sq_mulens.argmin(),full_chi_sq_mulens.shape)
uo0 = uo_grid[min_index[0]]
tE0 = tE_grid[min_index[1]]
chi_sq_mulens=np.min(full_chi_sq_mulens)
model_in_magnitude_mulens=full_model_in_magnitude[min_index[0],min_index[1]]
print "\n Grid search space is minimized at uo=",uo_grid[min_index[0]]," and tE=",tE_grid[min_index[1]],"\n"

In [ ]:
end_time_mlm = datetime.now()
print('Total Runtime: {0}'.format(end_time_mlm - start_time_mlm))
t= format(end_time_mlm - start_time_mlm)
(hours, minutes, seconds) = t.split(':')
hours_to_run_mlm = float(hours) + float(minutes) / 60 + float(seconds) / 3600

In [ ]:
ModelID_mlm= 'WFIRST_1827.dat_MulensModel' #Model ID containing both target ID and solution number e.g. [target]_[solution]
t0_mlm = to #Time of peak, t0 [days] - priority
sig_t0_mlm = to_grid[1]-to_grid[0] #Uncertainty in t0 [days] - priority
tE_mlm = tE0  #Einstein crossing time, tE [days] - priority
sig_tE_mlm = tE_grid[1]-tE_grid[0] #Uncertainty in tE [days] - priority
u0_mlm = uo0  #Minimum impact parameter, u0 [normalised by θE] - priority
sig_u0_mlm = uo_grid[1]-uo_grid[0] #Uncertainty in u0 - priority
rho_mlm = rho #Angular source size parameter, rho
sig_rho_mlm = rho_grid[1]-rho_grid[0] #Uncertainty on rho
piEE_mlm = 'None'  #Parallax parameter πE,E
sig_piEE_mlm = 'None'  #Uncertainty on πE,E
piEN_mlm = 'None'  #Parallax parameter πE,N
sig_piEN_mlm = 'None'  #Uncertainty on πE,N
fs_mlm = 'None' #Source flux, fs [counts/s] - priority
sig_fs_mlm = 'None' #Uncertainty in fs [counts/s] - priority
fb_mlm = 0 #Blend flux, fb [counts/s] - priority
sig_fb_mlm = 0 #Uncertainty in fb [counts/s] - priority
s_mlm = 10**(log_s) #Binary separation, s, [normalised by θE] - priority
sig_s_mlm = 10**(log_s_grid[1]-log_s_grid[0]) #Uncertainty on s - priority
q_mlm = 10**(log_q) #mlm_event.model.parameters.q #Mass ratio, q = M2/M1 - priority
sig_q_mlm = 10**(log_q_grid[1]-log_q_grid[0]) #Uncertainty on q - priority
alpha_mlm = alpha #mlm_event.model.parameters.alpha #Angle of lens motion, alpha - priority
sig_alpha_mlm = alpha_grid[1]-alpha_grid[0] #Uncertainty on alpha - priority
dsdt_mlm = 'None'  #Rate of change of s, ds/dt
sig_dsdt_mlm = 'None'  #Uncertainty on ds/dt
dadt_mlm =  'None' #Rate of change of alpha, dalpha/dt
sig_dadt_mlm = 'None'  #Uncertainty on dalpha/dt
t0_par_mlm = 'None'  #t0_par [days]
chisq_mlm = chi_sq_mulens #Chi squared of the fitted model
M1_mlm = 'None'  #Primary lens mass, M1 [Msolar]
sig_M1_mlm = 'None'  #Uncertainty on M1 [Msolar]
M2_mlm = 'None'  #Secondary lens mass, M2 [MJupiter] 
sig_M2_mlm = 'None'  #Uncertainty on M2 [MJupiter] 
DL_mlm = 'None'  #Distance to the lens, DL [kpc]
sig_DL_mlm = 'None'  #Uncertainty on DL [kpc]
DS_mlm = 'None'  #Distance to the source, DS [kpc]
sig_DS_mlm = 'None'  #Uncertainty on DS [kpc]
aperp_mlm = 'None'  #Projected separation of lens masses, aperp [AU]
sig_aperp_mlm = 'None'  #Uncertainty on aperp [AU]
t_fit_mlm = hours_to_run_mlm #Time taken to fit the lightcurve from data ingest to final output [hrs]

# MuLAN

In [ ]:
start_time_mln = datetime.now()

In [ ]:
cfgsetup = cp.SafeConfigParser()
cfgsetup.add_section('EventDescription')
cfgsetup.set('EventDescription', 'RA', '17h57m11.52s')
cfgsetup.set('EventDescription', 'DEC', '-29d07m03.36s')

In [ ]:
# Import the model
import muLAn.models.BLcontU as esbl_vbb

# Define the dates we want to compute the model (e.g. from 8350 to 9122)
n_dates = WFIRST_data[:,0].shape[0]
timeserie = np.linspace(to-100-2460000, to+100-2460000, n_dates)


full_chi_sq_mulan=np.zeros((grid_steps,grid_steps))
full_model_in_magnitude=np.zeros((grid_steps,grid_steps,WFIRST_data[:,0].shape[0]))

# Prepare dataset before fit: compute the flux
photometry = pd.DataFrame({'date': WFIRST_data[:,0] - 2460000,
                           'flux': 10**(-0.4*WFIRST_data[:,1])})
photometry['magnification'] = 1.0  ## initialization of the column magnification

#Run loop a small grid on different values of your choice (in this case, uo and tE):
for j,uo_i in enumerate(uo_grid):
    uo0=uo_i
    for k,tE_i in enumerate(tE_grid):
        tE0=tE_i
        
        # Define the parameters (depends on each model)
        # --> in this example, no parallax
        lens_params = dict({'u0': -uo0,
                            'tE': tE0,
                            't0': to-2460000,
                            'piEN': 0.0,
                            'piEE': 0.0,
                            'rho': rho,
                            's': s,
                            'q': q,
                            'alpha': np.pi-alpha,
                            'dalpha': 0.0,
                            'ds': 0.0
                           })
        
        # No lens orbital motion (dalpha=0, ds=0)
        tb = lens_params['t0']  # we choose t_binary = t0 here (see, e.g., Skowron et al. 2011)

        # We don't want to include microlens parallax in this fit
        Ds = dict({'N':np.zeros(n_dates), 'E':np.zeros(n_dates)})

        # Compute magnification at the date of the observations
        photometry['magnification'] = esbl_vbb.magnifcalc(photometry['date'].values, 
                                                          lens_params, Ds=Ds, tb=tb)

        # Determine the flux of the source star (fs) and the blend flux (fb)
        # By definition, flux_total(t) = fs * A(t) + fb
        # with A=magnification
        table_sorted = photometry.sort_values('magnification', inplace=False)
        x = table_sorted['magnification'].values
        y = table_sorted['flux'].values
        fs, fb, r_value, p_value, std_err = stats.linregress(x, y)

        # Derive the model in magnitudes
        model_in_magnitude1 = - 2.5 * np.log10(fs * photometry['magnification'].values + fb)
        
        full_chi_sq_mulan[j,k],_=chisquare(WFIRST_data[:,1],f_exp=model_in_magnitude1)
        full_model_in_magnitude[j,k,:]=model_in_magnitude1
        
        
        # Plot the magnification
        x = photometry['date']
        y = model_in_magnitude1
#         plt.figure()
#         plt.plot(WFIRST_data[:,0]-2460000,WFIRST_data[:,1],color='black')
#         plt.plot(x, y)
#         plt.xlabel('HJD-2460000')
#         plt.ylabel('Magnification')
#         plt.xlim(910,1000)
#         plt.gca().invert_yaxis()
#         plt.title('Plot with parameter 1: '+str(uo0)+' and parameter 2: '+str(tE0))
#         plt.show()


plt.imshow(full_chi_sq_mulan,origin='bottom',extent=[tE_grid[0],tE_grid[-1],uo_grid[0],uo_grid[-1]],aspect="auto")
plt.xlabel('tE',fontsize=20)
plt.ylabel('uo',fontsize=20)
plt.colorbar(label='Chi-squared')
plt.show()




In [ ]:
min_index=np.unravel_index(full_chi_sq_mulan.argmin(),full_chi_sq_mulan.shape)
uo0 = uo_grid[min_index[0]]
tE0 = tE_grid[min_index[1]]
chi_sq_mulan=np.min(full_chi_sq_mulan)
model_in_magnitude_mulan=full_model_in_magnitude[min_index[0],min_index[1]]
print "\n Grid search space is minimized at uo=",uo_grid[min_index[0]]," and tE=",tE_grid[min_index[1]],"\n"


In [ ]:
end_time_mln = datetime.now()
print('Total Runtime: {0}'.format(end_time_mln - start_time_mln))
t= format(end_time_mln - start_time_mln)
(hours, minutes, seconds) = t.split(':')
hours_to_run_mln = float(hours) + float(minutes) / 60 + float(seconds) / 3600

In [ ]:
ModelID_mln= 'WFIRST_1827.dat_Mulan' #Model ID containing both target ID and solution number e.g. [target]_[solution]
t0_mln = to #Time of peak, t0 [days] - priority
sig_t0_mln = to_grid[1]-to_grid[0] #Uncertainty in t0 [days] - priority
tE_mln = tE0  #Einstein crossing time, tE [days] - priority
sig_tE_mln = tE_grid[1]-tE_grid[0] #Uncertainty in tE [days] - priority
u0_mln = uo0  #Minimum impact parameter, u0 [normalised by θE] - priority
sig_u0_mln = uo_grid[1]-uo_grid[0] #Uncertainty in u0 - priority
rho_mln = rho #Angular source size parameter, rho
sig_rho_mln = rho_grid[1]-rho_grid[0] #Uncertainty on rho
piEE_mln = 'None'  #Parallax parameter πE,E
sig_piEE_mln = 'None'  #Uncertainty on πE,E
piEN_mln = 'None'  #Parallax parameter πE,N
sig_piEN_mln = 'None'  #Uncertainty on πE,N
fs_mln = 'None' #Source flux, fs [counts/s] - priority
sig_fs_mln = 'None' #Uncertainty in fs [counts/s] - priority
fb_mln = 0 #Blend flux, fb [counts/s] - priority
sig_fb_mln = 0 #Uncertainty in fb [counts/s] - priority
s_mln = 10**(log_s) #Binary separation, s, [normalised by θE] - priority
sig_s_mln = 10**(log_s_grid[1]-log_s_grid[0]) #Uncertainty on s - priority
q_mln = 10**(log_q) #mlm_event.model.parameters.q #Mass ratio, q = M2/M1 - priority
sig_q_mln = 10**(log_q_grid[1]-log_q_grid[0]) #Uncertainty on q - priority
alpha_mln = alpha #mlm_event.model.parameters.alpha #Angle of lens motion, alpha - priority
sig_alpha_mln = alpha_grid[1]-alpha_grid[0] #Uncertainty on alpha - priority
dsdt_mln = 'None'  #Rate of change of s, ds/dt
sig_dsdt_mln = 'None'  #Uncertainty on ds/dt
dadt_mln =  'None' #Rate of change of alpha, dalpha/dt
sig_dadt_mln = 'None'  #Uncertainty on dalpha/dt
t0_par_mln = 'None'  #t0_par [days]
chisq_mln = chi_sq_mulan #Chi squared of the fitted model
M1_mln = 'None'  #Primary lens mass, M1 [Msolar]
sig_M1_mln = 'None'  #Uncertainty on M1 [Msolar]
M2_mln = 'None'  #Secondary lens mass, M2 [MJupiter] 
sig_M2_mln = 'None'  #Uncertainty on M2 [MJupiter] 
DL_mln = 'None'  #Distance to the lens, DL [kpc]
sig_DL_mln = 'None'  #Uncertainty on DL [kpc]
DS_mln = 'None'  #Distance to the source, DS [kpc]
sig_DS_mln = 'None'  #Uncertainty on DS [kpc]
aperp_mln = 'None'  #Projected separation of lens masses, aperp [AU]
sig_aperp_mln = 'None'  #Uncertainty on aperp [AU]
t_fit_mln = hours_to_run_mln #Time taken to fit the lightcurve from data ingest to final output [hrs]

In [ ]:
##Just to compare the three models, if you want to see the difference. Not doing a perfect fit, for now.
plt.figure()
plt.plot(WFIRST_data[:,0]-2460000,WFIRST_data[:,1],color='black',label='WFIRST simulated data')
plt.plot(times[1:]-2460000, model_in_magnitude_pylima,color='red',label='pyLIMA, chi-sq='+str(np.round(chi_sq_pylima,3)))
plt.plot(times-2460000, model_in_magnitude_mulens,color='blue',label='MuLensModel, chi-sq='+str(np.round(chi_sq_mulens,3)))
plt.plot(times-2460000, model_in_magnitude_mulan,color='green',label='muLAn, chi-sq='+str(np.round(chi_sq_mulens,3)))
#plt.xlim(2460880,2461000)
plt.xlim(920,990)
plt.gca().invert_yaxis()
plt.ylabel('Magnitude')
plt.xlabel('HJD-2460000')
plt.legend()
plt.show()

In [ ]:
##Zooming in
plt.figure()
plt.plot(WFIRST_data[:,0]-2460000,WFIRST_data[:,1],color='black',label='WFIRST simulated data')
plt.plot(times[1:]-2460000, model_in_magnitude_pylima,color='red',label='pyLIMA, chi-sq='+str(np.round(chi_sq_pylima,3)))
plt.plot(times-2460000, model_in_magnitude_mulens,color='blue',label='MuLensModel, chi-sq='+str(np.round(chi_sq_mulens,3)))
plt.plot(times-2460000, model_in_magnitude_mulan,color='green',label='muLAn, chi-sq='+str(np.round(chi_sq_mulens,3)))
#plt.xlim(920,990)
plt.xlim(982,985)
plt.gca().invert_yaxis()
plt.ylabel('Magnitude')
plt.xlabel('HJD-2460000')
plt.legend()
plt.savefig(local_path+'data_challenge/models.pdf')
plt.show()

In [ ]:
Class = 'GLplanet' #Classification
f = open(local_path+'data_challenge/table1.dat', 'a')
f.write('\n%s %s %s %s %s %s %s %s %s %s %s %s %s %s %s %s %s %s %s %s '
          '%s %s %s %s %s %s %s %s %s %s %s %s %s %s %s %s %s %s %s %s %s' %
        (ModelID_pl, Class, t0_pl, sig_t0_pl, tE_pl, sig_tE_pl, u0_pl, sig_u0_pl, rho_pl, sig_rho_pl, piEE_pl,
         sig_piEE_pl, piEN_pl, sig_piEN_pl, fs_pl, sig_fs_pl, fb_pl, sig_fb_pl, s_pl, sig_s_pl, q_pl, sig_q_pl, 
         alpha_pl, sig_alpha_pl, dsdt_pl, sig_dsdt_pl, dadt_pl, sig_dadt_pl, t0_par_pl, chisq_pl, M1_pl, 
         sig_M1_pl, M2_pl, sig_M2_pl, DL_pl, sig_DL_pl, DS_pl, sig_DS_pl, aperp_pl, sig_aperp_pl, t_fit_pl))
f.write('\n%s %s %s %s %s %s %s %s %s %s %s %s %s %s %s %s %s %s %s %s '
          '%s %s %s %s %s %s %s %s %s %s %s %s %s %s %s %s %s %s %s %s %s' %
        (ModelID_mlm, Class, t0_mlm, sig_t0_mlm, tE_mlm, sig_tE_mlm, u0_mlm, sig_u0_mlm, rho_mlm, sig_rho_mlm, piEE_mlm,
         sig_piEE_mlm, piEN_mlm, sig_piEN_mlm, fs_mlm, sig_fs_mlm, fb_mlm, sig_fb_mlm, s_mlm, sig_s_mlm, q_mlm, sig_q_mlm, 
         alpha_mlm, sig_alpha_mlm, dsdt_mlm, sig_dsdt_mlm, dadt_mlm, sig_dadt_mlm, t0_par_mlm, chisq_mlm, M1_mlm, 
         sig_M1_mlm, M2_mlm, sig_M2_mlm, DL_mlm, sig_DL_mlm, DS_mlm, sig_DS_mlm, aperp_mlm, sig_aperp_mlm, t_fit_mlm))
f.write('\n%s %s %s %s %s %s %s %s %s %s %s %s %s %s %s %s %s %s %s %s '
          '%s %s %s %s %s %s %s %s %s %s %s %s %s %s %s %s %s %s %s %s %s' %
        (ModelID_mln, Class, t0_mln, sig_t0_mln, tE_mln, sig_tE_mln, u0_mln, sig_u0_mln, rho_mln, sig_rho_mln, piEE_mln,
         sig_piEE_mln, piEN_mln, sig_piEN_mln, fs_mln, sig_fs_mln, fb_mln, sig_fb_mln, s_mln, sig_s_mln, q_mln, sig_q_mln, 
         alpha_mln, sig_alpha_mln, dsdt_mln, sig_dsdt_mln, dadt_mln, sig_dadt_mln, t0_par_mln, chisq_mln, M1_mln, 
         sig_M1_mln, M2_mln, sig_M2_mln, DL_mln, sig_DL_mln, DS_mln, sig_DS_mln, aperp_mln, sig_aperp_mln, t_fit_mln))
f.close()